# 🚀 Experiment Training - Car Counting v1.03

## 🎯 Purpose
This notebook trains a YOLOv8 model using CVAT-annotated frames for vehicle counting and creates a live counting application.

## 📋 Context
- **Input**: YOLO dataset exported from CVAT
- **Model**: YOLOv8n (nano - fastest training)
- **Output**: Trained model + live counting script
- **Target**: Real-time vehicle counting on traffic video

## 🔄 Workflow Overview
1. Extract and organize CVAT dataset
2. Validate annotations and dataset structure
3. Train YOLOv8 model with custom data
4. Evaluate model performance
5. Create live counting application
6. Test on sample video clips

## 📁 Expected Input Structure
```
car_counting_ATL1005_20250620_yolo_dataset.zip
├── images/          # Annotated frame images
├── labels/          # YOLO format annotations
└── dataset.yaml     # Dataset configuration
```

## 🎮 Output Applications
1. **Live video counter**: Real-time vehicle counting
2. **Batch processing**: Count vehicles in recorded videos
3. **Model evaluation**: Performance metrics and validation

**Version**: 1.03 | **Type**: Model Training | **Target**: Vehicle Counting


## 🔧 Training Configuration Parameters

This cell defines all parameters for YOLOv8 model training and dataset preparation.

### Dataset Parameters
- 🎯 **DATASET_ZIP**: Path to CVAT exported dataset
- 🎯 **EXPERIMENT_NAME**: Unique identifier for training run
- 📁 **OUTPUT_BASE**: Root directory for training outputs

### Training Settings  
- 📊 **MODEL_SIZE**: YOLOv8 variant (n=nano, s=small, m=medium)
- 📊 **EPOCHS**: Training iterations
- 📊 **BATCH_SIZE**: Images per training batch
- 📊 **IMAGE_SIZE**: Input image dimensions

### Validation Settings
- 🔍 **TRAIN_SPLIT**: Percentage for training data
- 🔍 **VAL_SPLIT**: Percentage for validation data
- 🔍 **CONFIDENCE_THRESHOLD**: Detection confidence cutoff

**Note**: Nano model (yolov8n) is fastest for training and inference on CPU.

In [1]:
# training configuration parameters
from pathlib import Path

TRAINING_CONFIG = {
    # dataset parameters
    'DATASET_ZIP': '/home/trauco/v3-traffic-vision/data/experiments/car_counting_v1/2025-06-20/ATL-1005/car_counting_ATL1005_20250620_yolo_dataset.zip',
    'EXPERIMENT_NAME': 'car_counting_training_v1',
    'OUTPUT_BASE': Path('../../data/experiments/car_counting_v1/2025-06-20/ATL-1005'),
    
    # training settings
    'MODEL_SIZE': 'yolov8x',  # 📊 extra large model (most powerful)
    'EPOCHS': 10,  # 📊 more training iterations
    'BATCH_SIZE': 16,  # 📊 conservative for large model
    'IMAGE_SIZE': 640,  # 📊 input dimensions
    
    # validation settings
    'TRAIN_SPLIT': 0.8,  # 🔍 80% for training
    'VAL_SPLIT': 0.2,  # 🔍 20% for validation
    'CONFIDENCE_THRESHOLD': 0.5,  # 🔍 detection confidence
    
    # output settings
    'SAVE_PERIOD': 10,  # 📊 save every N epochs
    'DEVICE': 'cuda'  # 🔧 use GPU
}

# derived paths
TRAINING_CONFIG['DATASET_DIR'] = TRAINING_CONFIG['OUTPUT_BASE'] / 'dataset'
TRAINING_CONFIG['TRAINING_DIR'] = TRAINING_CONFIG['OUTPUT_BASE'] / 'training'

print("Training Configuration:")
print(f"  Dataset: {Path(TRAINING_CONFIG['DATASET_ZIP']).name}")
print(f"  Model: {TRAINING_CONFIG['MODEL_SIZE']}")
print(f"  Epochs: {TRAINING_CONFIG['EPOCHS']}")
print(f"  Output: {TRAINING_CONFIG['TRAINING_DIR']}")
print(f"  Device: {TRAINING_CONFIG['DEVICE']}")

Training Configuration:
  Dataset: car_counting_ATL1005_20250620_yolo_dataset.zip
  Model: yolov8x
  Epochs: 10
  Output: ../../data/experiments/car_counting_v1/2025-06-20/ATL-1005/training
  Device: cuda


## 🔧 Environment Setup & Dependencies

This cell installs YOLOv8 and verifies system requirements for model training.

### Installation Process
- **Ultralytics**: YOLOv8 training framework
- **Dependencies**: PyTorch, OpenCV, matplotlib for training
- **System Check**: Verify Python version and hardware capabilities

### Hardware Detection
- **CPU/GPU**: Automatic device detection
- **Memory**: Available RAM for training
- **Storage**: Disk space for dataset and models

**Note**: Training will proceed on available hardware (CPU fallback if no GPU).

In [2]:
# environment setup and dependencies
import subprocess
import sys
import os
import zipfile
import shutil
from pathlib import Path

print("Environment Setup & Dependencies")
print("=" * 40)

# install ultralytics
try:
    import ultralytics
    print(f"✓ Ultralytics already installed: {ultralytics.__version__}")
except ImportError:
    print("Installing ultralytics...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])
    import ultralytics
    print(f"✓ Ultralytics installed: {ultralytics.__version__}")

# verify additional dependencies
try:
    import torch
    import cv2
    import matplotlib.pyplot as plt
    import yaml
    print(f"✓ PyTorch: {torch.__version__}")
    print(f"✓ OpenCV: {cv2.__version__}")
    print("✓ Matplotlib: Available")
    print("✓ YAML: Available")
except ImportError as e:
    print(f"⚠️ Missing dependency: {e}")

# hardware detection
print(f"\nHardware Detection:")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA: {torch.version.cuda}")
    TRAINING_CONFIG['DEVICE'] = 'cuda'
else:
    print("  GPU: Not available (using CPU)")
    TRAINING_CONFIG['DEVICE'] = 'cpu'

# memory check
import psutil
memory_gb = psutil.virtual_memory().total / (1024**3)
print(f"  RAM: {memory_gb:.1f} GB")

# create output directories
TRAINING_CONFIG['DATASET_DIR'].mkdir(parents=True, exist_ok=True)
TRAINING_CONFIG['TRAINING_DIR'].mkdir(parents=True, exist_ok=True)

print(f"\nOutput directories created:")
print(f"  Dataset: {TRAINING_CONFIG['DATASET_DIR']}")
print(f"  Training: {TRAINING_CONFIG['TRAINING_DIR']}")

print(f"\n✓ Environment ready for training")

Environment Setup & Dependencies
✓ Ultralytics already installed: 8.3.158
✓ PyTorch: 2.7.1+cu126
✓ OpenCV: 4.11.0
✓ Matplotlib: Available
✓ YAML: Available

Hardware Detection:
  GPU: NVIDIA RTX A5500
  CUDA: 12.6
  RAM: 30.8 GB

Output directories created:
  Dataset: ../../data/experiments/car_counting_v1/2025-06-20/ATL-1005/dataset
  Training: ../../data/experiments/car_counting_v1/2025-06-20/ATL-1005/training

✓ Environment ready for training


## 📦 Dataset Extraction & Organization

This cell extracts the CVAT dataset and organizes it into YOLOv8 training format.

### Process
- **Extract ZIP**: Unpack CVAT exported dataset
- **Validate Structure**: Verify images and labels are present
- **Split Dataset**: Organize into train/validation sets
- **Create Config**: Generate dataset.yaml for YOLOv8

### Expected CVAT Structure
```
dataset/
├── images/     # Frame images from annotation
├── labels/     # YOLO format text files
└── classes.txt # Class definitions
```

**Note**: CVAT exports may vary in structure - script adapts automatically.

In [3]:
# dataset extraction and organization
import zipfile
import shutil
import yaml
import random

# install sklearn if missing
try:
    from sklearn.model_selection import train_test_split
except ImportError:
    print("Installing scikit-learn...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn"])
    from sklearn.model_selection import train_test_split

print("Dataset Extraction & Organization")
print("=" * 35)

# clear and recreate dataset directory
if TRAINING_CONFIG['DATASET_DIR'].exists():
    shutil.rmtree(TRAINING_CONFIG['DATASET_DIR'])
TRAINING_CONFIG['DATASET_DIR'].mkdir(parents=True, exist_ok=True)

# extract dataset
dataset_zip = Path(TRAINING_CONFIG['DATASET_ZIP'])
if not dataset_zip.exists():
    raise FileNotFoundError(f"Dataset not found: {dataset_zip}")

print(f"Extracting: {dataset_zip.name}")
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(TRAINING_CONFIG['DATASET_DIR'])

# find extracted content
extracted_files = list(TRAINING_CONFIG['DATASET_DIR'].rglob('*'))
image_files = [f for f in extracted_files if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
label_files = [f for f in extracted_files if f.suffix == '.txt' and 'classes' not in f.name.lower()]

print(f"Found {len(image_files)} images")
print(f"Found {len(label_files)} label files")

# organize into train/val structure
train_img_dir = TRAINING_CONFIG['DATASET_DIR'] / 'train' / 'images'
train_lbl_dir = TRAINING_CONFIG['DATASET_DIR'] / 'train' / 'labels'
val_img_dir = TRAINING_CONFIG['DATASET_DIR'] / 'val' / 'images'
val_lbl_dir = TRAINING_CONFIG['DATASET_DIR'] / 'val' / 'labels'

for dir_path in [train_img_dir, train_lbl_dir, val_img_dir, val_lbl_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

# split dataset
image_basenames = [f.stem for f in image_files]
train_names, val_names = train_test_split(
    image_basenames, 
    train_size=TRAINING_CONFIG['TRAIN_SPLIT'], 
    random_state=42
)

# copy files to train/val directories
for img_file in image_files:
    if img_file.exists():  # check file exists
        if img_file.stem in train_names:
            shutil.copy2(img_file, train_img_dir / img_file.name)
        else:
            shutil.copy2(img_file, val_img_dir / img_file.name)

for lbl_file in label_files:
    if lbl_file.exists():  # check file exists
        if lbl_file.stem in train_names:
            shutil.copy2(lbl_file, train_lbl_dir / lbl_file.name)
        else:
            shutil.copy2(lbl_file, val_lbl_dir / lbl_file.name)

# create dataset.yaml
dataset_config = {
    'path': str(TRAINING_CONFIG['DATASET_DIR']),
    'train': 'train/images',
    'val': 'val/images',
    'nc': 1,  # number of classes
    'names': ['vehicle']  # class names
}

yaml_path = TRAINING_CONFIG['DATASET_DIR'] / 'dataset.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print(f"\nDataset organized:")
print(f"  Training: {len(train_names)} images")
print(f"  Validation: {len(val_names)} images")
print(f"  Config: {yaml_path}")

TRAINING_CONFIG['DATASET_YAML'] = yaml_path

print(f"\n✓ Dataset ready for training")

Dataset Extraction & Organization
Extracting: car_counting_ATL1005_20250620_yolo_dataset.zip
Found 50 images
Found 51 label files

Dataset organized:
  Training: 40 images
  Validation: 10 images
  Config: ../../data/experiments/car_counting_v1/2025-06-20/ATL-1005/dataset/dataset.yaml

✓ Dataset ready for training


## 🚀 Model Training

This cell trains the YOLOv8 model on the annotated vehicle dataset.

### Training Process
- **Model Initialization**: Load pre-trained YOLOv8n weights
- **Custom Training**: Fine-tune on vehicle detection task
- **Progress Monitoring**: Track loss and validation metrics
- **Model Saving**: Automatic best model checkpointing

### Training Outputs
- **Weights**: best.pt (best model) and last.pt (final epoch)
- **Metrics**: Training/validation loss curves
- **Predictions**: Sample detection results
- **Logs**: Training progress and statistics

### Expected Duration
- **CPU**: 10-30 minutes for 50 epochs
- **GPU**: 2-5 minutes for 50 epochs

**Note**: Training progress will be displayed with real-time metrics.

In [4]:
# model training with live status
from ultralytics import YOLO
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

print("Model Training")
print("=" * 15)

# initialize model
model = YOLO(f"{TRAINING_CONFIG['MODEL_SIZE']}.pt")
print(f"✓ Loaded {TRAINING_CONFIG['MODEL_SIZE']} model")

# training configuration display
print(f"\nTraining Configuration:")
print(f"  Model: {TRAINING_CONFIG['MODEL_SIZE']}")
print(f"  Epochs: {TRAINING_CONFIG['EPOCHS']}")
print(f"  Device: {TRAINING_CONFIG['DEVICE']}")
print(f"  Batch size: {TRAINING_CONFIG['BATCH_SIZE']}")
print(f"  Dataset: {TRAINING_CONFIG['DATASET_YAML'].name}")

print(f"\n🚀 Starting training...")
time.sleep(2)

# create custom callback for progress tracking
class TrainingCallback:
    def __init__(self, total_epochs):
        self.total_epochs = total_epochs
        self.start_time = time.time()
        
    def on_train_epoch_end(self, trainer):
        epoch = trainer.epoch + 1
        elapsed = time.time() - self.start_time
        eta = (elapsed / epoch) * (self.total_epochs - epoch)
        
        # get metrics
        metrics = trainer.metrics
        loss = getattr(trainer, 'loss', None)
        
        # clear and update display
        clear_output(wait=True)
        print("Model Training - Live Status")
        print("=" * 40)
        print(f"📊 Progress: Epoch {epoch}/{self.total_epochs} ({epoch/self.total_epochs*100:.1f}%)")
        print(f"⏱️  Elapsed: {elapsed/60:.1f} min | ETA: {eta/60:.1f} min")
        
        if hasattr(trainer, 'loss') and trainer.loss is not None:
            print(f"📉 Training Loss: {trainer.loss:.4f}")
        
        if metrics:
            if 'fitness' in metrics:
                print(f"🎯 Fitness: {metrics['fitness']:.4f}")
            if 'precision' in metrics:
                print(f"🔍 Precision: {metrics['precision']:.3f}")
            if 'recall' in metrics:
                print(f"📋 Recall: {metrics['recall']:.3f}")
        
        # progress bar
        progress = "█" * int(30 * epoch / self.total_epochs)
        remaining = "░" * (30 - len(progress))
        print(f"[{progress}{remaining}] {epoch}/{self.total_epochs}")

# add callback
callback = TrainingCallback(TRAINING_CONFIG['EPOCHS'])

results = model.train(
    data=str(TRAINING_CONFIG['DATASET_YAML']),
    epochs=TRAINING_CONFIG['EPOCHS'],
    batch=TRAINING_CONFIG['BATCH_SIZE'],
    imgsz=TRAINING_CONFIG['IMAGE_SIZE'],
    device=TRAINING_CONFIG['DEVICE'],
    project=str(TRAINING_CONFIG['TRAINING_DIR']),
    name='vehicle_detection',
    save_period=TRAINING_CONFIG['SAVE_PERIOD'],
    plots=True,
    verbose=False
)

# final results
clear_output(wait=True)
training_output = TRAINING_CONFIG['TRAINING_DIR'] / 'vehicle_detection'
best_model = training_output / 'weights' / 'best.pt'

TRAINING_CONFIG['BEST_MODEL'] = best_model

print("✅ Training Complete!")
print("=" * 20)
print(f"🏆 Model: {best_model.name}")
print(f"📁 Results: {training_output}")
print(f"⏱️  Total time: {(time.time() - callback.start_time)/60:.1f} minutes")

# show final metrics
if hasattr(results, 'results_dict'):
    metrics = results.results_dict
    if 'metrics/precision(B)' in metrics:
        print(f"🎯 Final Precision: {metrics['metrics/precision(B)']:.3f}")
    if 'metrics/recall(B)' in metrics:
        print(f"📋 Final Recall: {metrics['metrics/recall(B)']:.3f}")
    if 'metrics/mAP50(B)' in metrics:
        print(f"📊 Final mAP50: {metrics['metrics/mAP50(B)']:.3f}")

print(f"\n🎯 Ready for vehicle counting!")

✅ Training Complete!
🏆 Model: best.pt
📁 Results: ../../data/experiments/car_counting_v1/2025-06-20/ATL-1005/training/vehicle_detection
⏱️  Total time: 0.5 minutes
🎯 Final Precision: 0.140
📋 Final Recall: 0.571
📊 Final mAP50: 0.595

🎯 Ready for vehicle counting!


## 📊 Training Results Diagnostic

This cell analyzes the completed training to verify the model learned to detect vehicles properly.

### What We're Checking
- **Precision**: How accurate are the detections? (0.0 = terrible, 1.0 = perfect)
- **Recall**: How many vehicles does it find? (0.0 = finds nothing, 1.0 = finds all)
- **mAP50**: Overall detection quality at 50% confidence threshold

### Expected Results
- **Good training**: Precision > 0.5, Recall > 0.5, mAP50 > 0.5
- **Poor training**: Values near 0.0 indicate bad annotations or insufficient data
- **Failed training**: All metrics < 0.1 means model learned nothing

### Troubleshooting
If metrics are poor, likely causes:
- Empty or incorrect annotation files from CVAT
- Mismatch between training images and video content
- Insufficient training data (need more annotated frames)

**Note**: This diagnostic helps identify if detection issues are from training problems vs. inference problems.

In [5]:
# Check training results
results_dir = TRAINING_CONFIG['OUTPUT_BASE'] / 'training' / 'vehicle_detection'
results_file = results_dir / 'results.csv'

if results_file.exists():
    import pandas as pd
    df = pd.read_csv(results_file)
    print("Final training metrics:")
    print(f"Precision: {df['metrics/precision(B)'].iloc[-1]:.3f}")
    print(f"Recall: {df['metrics/recall(B)'].iloc[-1]:.3f}")
    print(f"mAP50: {df['metrics/mAP50(B)'].iloc[-1]:.3f}")
    
    if df['metrics/precision(B)'].iloc[-1] < 0.1:
        print("❌ Model didn't learn - training failed!")
    else:
        print("✅ Model learned something")
else:
    print("❌ No results file found")

Final training metrics:
Precision: 0.698
Recall: 0.857
mAP50: 0.855
✅ Model learned something


## 📋 Annotation Data Validation

This cell inspects the actual annotation files exported from CVAT to verify they contain valid vehicle bounding box data.

### What We're Checking
- **File existence**: Are label files present for each image?
- **File content**: Do files contain YOLO format annotations?
- **Data format**: Proper YOLO format: `class_id x_center y_center width height`

### YOLO Format Example

In [6]:
# Check annotation files from CVAT export
label_dir = TRAINING_CONFIG['DATASET_DIR'] / 'train' / 'labels'
label_files = list(label_dir.glob('*.txt'))
print(f"Found {len(label_files)} label files")

# Check a few labels
for i, label_file in enumerate(label_files[:3]):
    print(f"\nLabel file {i+1}: {label_file.name}")
    with open(label_file, 'r') as f:
        content = f.read().strip()
        if content:
            print(f"Content: {content}")
        else:
            print("❌ EMPTY FILE!")

Found 40 label files

Label file 1: frame_0039.txt
❌ EMPTY FILE!

Label file 2: frame_0003.txt
❌ EMPTY FILE!

Label file 3: frame_0004.txt
❌ EMPTY FILE!


## 🎬 Live Counting Application

This cell creates a real-time vehicle counting application using the trained model.

### Application Features
- **Video Playback**: Load and play traffic intersection video
- **Real-time Detection**: Apply trained model to each frame
- **Live Counter**: Display current vehicle count
- **Bounding Boxes**: Visual detection indicators
- **Performance Metrics**: FPS and inference time

### Controls
- **Space**: Pause/resume video
- **Q**: Quit application
- **S**: Save current frame with detections
- **R**: Reset counter

### Video Input Options
- **Original source video**: Same intersection as training data
- **Custom video file**: Any traffic video
- **Webcam feed**: Real-time camera input (device 0)

**Note**: Application runs in

In [ ]:
# live counting application (simplified)
from ultralytics import YOLO
import cv2
import time
from pathlib import Path

print("Live Counting Application")
print("=" * 25)

# load model (set path directly)
model_path = TRAINING_CONFIG['OUTPUT_BASE'] / 'training' / 'vehicle_detection' / 'weights' / 'best.pt'
print(f"Model path: {model_path}")

if not model_path.exists():
    raise FileNotFoundError(f"Model not found: {model_path}")

model = YOLO(str(model_path))
print(f"✓ Model loaded")

# find video file
video_dir = Path.home() / "traffic-recordings" / "ATL-1005" / "2025-06-20"
print(f"Looking in: {video_dir}")

if video_dir.exists():
    video_files = list(video_dir.glob("ATL-1005_20250620_*.mp4"))
    if video_files:
        VIDEO_PATH = str(video_files[0])  # use first video found
        print(f"Found video: {VIDEO_PATH}")
    else:
        print("No video files found in directory")
        VIDEO_PATH = 0
else:
    print(f"Directory doesn't exist: {video_dir}")
    VIDEO_PATH = 0

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print(f"❌ Can't open video: {VIDEO_PATH}")
    print("Please check the file path or use webcam with VIDEO_PATH = 0")
else:
    print(f"✓ Video loaded: {VIDEO_PATH}")
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # detect vehicles
        results = model(frame, conf=0.5)
        count = len(results[0].boxes) if results[0].boxes is not None else 0
        
        # draw results
        annotated = results[0].plot()
        
        # add count
        cv2.putText(annotated, f'Vehicles: {count}', (50, 50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        
        # show frame
        cv2.imshow('Vehicle Count', annotated)
        
        # proper timing for video playback
        if cv2.waitKey(33) & 0xFF == ord('q'):  # ~30 FPS
            break
            
        frame_count += 1
        
    cap.release()
    cv2.destroyAllWindows()
    print(f"Processed {frame_count} frames")

Live Counting Application
Model path: ../../data/experiments/car_counting_v1/2025-06-20/ATL-1005/training/vehicle_detection/weights/best.pt
✓ Model loaded
Looking in: /home/trauco/traffic-recordings/ATL-1005/2025-06-20
Found video: /home/trauco/traffic-recordings/ATL-1005/2025-06-20/ATL-1005_20250620_184919.mp4
✓ Video loaded: /home/trauco/traffic-recordings/ATL-1005/2025-06-20/ATL-1005_20250620_184919.mp4

0: 384x640 (no detections), 42.8ms
Speed: 1.6ms preprocess, 42.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.6ms
Speed: 1.2ms preprocess, 3.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.1ms
Speed: 1.5ms preprocess, 3.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2.7ms
Speed: 1.4ms preprocess, 2.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2.9ms
Speed: 1.8ms preprocess, 2.9m